In [1]:
import numpy as np
import pandas as pd
import os

In [13]:
source_path = '../../datasets/data'

In [21]:
7632 / 24

318.0

In [14]:
print(os.listdir(source_path))

['cooling_demand.csv', 'electricity_demand.csv', 'electricity_price.csv', 'heating_demand.csv', 'solar_radiation.csv', 'T_a.csv']


In [ ]:
price       =   pd.read_csv(source_path+'/electricity_price.csv', index_col=0).to_numpy()
solar_radiation =   pd.read_csv(source_path+'/solar_radiation.csv', index_col=0).to_numpy()
e_demand    =   pd.read_csv(source_path+'/electricity_demand.csv', index_col=0).to_numpy()
h_demand    =   pd.read_csv(source_path+'/heating_demand.csv', index_col=0).to_numpy()
c_demand    =   pd.read_csv(source_path+'/cooling_demand.csv', index_col=0).to_numpy()
T_a       =   pd.read_csv(source_path+'/T_a.csv', index_col=0).to_numpy()

month = np.arange(1, 13)
days  = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
train_price = []
train_solar_radiation = []
train_e_demand = []
train_h_demand = []
train_c_demand = []

for m in month:
    days_in_month = days[m-1]
    # 获取训练集和测试集的索引
    train_indices = np.arange(0, days_in_month - 10)  # 除去最后10天
    test_indices = np.arange(days_in_month - 10, days_in_month)  # 最后10天

    train_price.append(price[train_indices])

train_price = np.vstack(train_price)

[[0.2421 0.2354 0.2284 ... 0.2869 0.2837 0.2803]
 [0.2379 0.2251 0.2169 ... 0.2588 0.2404 0.2272]
 [0.2358 0.2145 0.2029 ... 0.277  0.2629 0.2395]
 ...
 [0.2576 0.2505 0.2486 ... 0.2965 0.2819 0.2622]
 [0.2292 0.226  0.2195 ... 0.4296 0.3685 0.3562]
 [0.3275 0.3172 0.3107 ... 0.3689 0.3617 0.3499]]
[[0.2421 0.2354 0.2284 ... 0.2869 0.2837 0.2803]
 [0.2379 0.2251 0.2169 ... 0.2588 0.2404 0.2272]
 [0.2358 0.2145 0.2029 ... 0.277  0.2629 0.2395]
 ...
 [0.2014 0.1819 0.1745 ... 0.2546 0.2378 0.2202]
 [0.212  0.1986 0.1947 ... 0.2275 0.2136 0.1981]
 [0.2113 0.1964 0.1887 ... 0.2261 0.2083 0.1978]]


In [6]:
file_path = 'datasets\citylearn_challenge_2021\weather.csv'
file = pd.read_csv(file_path)
file

FileNotFoundError: [Errno 2] No such file or directory: 'datasets\\citylearn_challenge_2021\\weather.csv'

In [42]:
T_a = file.iloc[:, 0]
T_a  # Series

0        9.40
1        7.61
2        8.11
3        8.36
4        8.78
         ... 
35035    4.75
35036    3.55
35037    2.54
35038    1.45
35039    0.75
Name: Outdoor Drybulb Temperature [C], Length: 35040, dtype: float64

In [43]:
time_step = 365 * 24 * 60 / len(T_a)  # 按照每隔15min的方式测量记录

In [44]:
# 创建时间索引，从特定时间开始，设置频率为每分钟
start_time = '2023-01-01 00:00:00'
time_index = pd.date_range(start=start_time, periods=len(T_a), freq='15T')  # 每分钟一个数据点

# 将时间索引赋值给 Series
T_a.index = time_index

# 打印结果
print(T_a)

2023-01-01 00:00:00    9.40
2023-01-01 00:15:00    7.61
2023-01-01 00:30:00    8.11
2023-01-01 00:45:00    8.36
2023-01-01 01:00:00    8.78
                       ... 
2023-12-31 22:45:00    4.75
2023-12-31 23:00:00    3.55
2023-12-31 23:15:00    2.54
2023-12-31 23:30:00    1.45
2023-12-31 23:45:00    0.75
Freq: 15T, Name: Outdoor Drybulb Temperature [C], Length: 35040, dtype: float64


In [45]:
T_a_hourly = T_a.resample('1H').mean()
T_a_hourly

2023-01-01 00:00:00     8.3700
2023-01-01 01:00:00     9.4525
2023-01-01 02:00:00    11.6050
2023-01-01 03:00:00    14.6150
2023-01-01 04:00:00    12.6600
                        ...   
2023-12-31 19:00:00     7.0700
2023-12-31 20:00:00     9.9075
2023-12-31 21:00:00    10.1525
2023-12-31 22:00:00     5.9425
2023-12-31 23:00:00     2.0725
Freq: H, Name: Outdoor Drybulb Temperature [C], Length: 8760, dtype: float64

In [48]:
T_a_hourly = T_a_hourly.reset_index(drop=True)
T_a_hourly = pd.DataFrame(T_a_hourly.to_numpy().reshape(int(len(T_a_hourly)/24), 24))
T_a_hourly

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,8.3700,9.4525,11.6050,14.6150,12.6600,13.3000,13.3875,13.2450,13.9325,12.9850,...,2.9750,5.7400,3.9800,2.3125,0.8250,2.6350,8.2775,13.2850,12.0125,11.1850
1,11.5100,12.3350,14.7900,19.7775,16.1800,7.9025,1.9875,-0.4975,0.1400,1.6125,...,-1.6825,0.5650,-0.0350,-1.2575,-2.0200,-0.9775,3.9575,7.8125,6.3000,6.0600
2,5.6950,5.3050,11.8450,16.4475,13.6250,11.6325,10.9825,11.5025,13.8825,17.6025,...,20.9325,21.4000,16.2975,11.9475,7.0700,5.9975,15.5150,18.3875,14.1025,11.7925
3,13.7375,14.2725,16.8300,17.4425,17.4000,14.7650,11.3800,8.7175,13.2350,17.1200,...,10.0975,11.6275,9.0225,6.1025,3.1475,1.8950,9.3200,14.8025,12.2900,9.9650
4,10.3875,10.0850,12.7925,14.9900,10.7975,8.0025,4.8625,2.3450,9.5800,13.6425,...,13.9725,16.4700,14.2425,9.4700,7.2375,5.0475,13.9250,18.8050,16.1450,13.0950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,11.2700,11.5650,13.9975,16.2975,12.4575,9.2750,7.0650,3.0325,11.0625,16.0425,...,15.0000,17.0675,14.4900,12.4200,11.5550,10.5475,9.4800,9.1275,9.1975,8.6325
361,7.9475,7.9025,9.9375,12.4975,12.4125,14.4500,17.9775,20.5300,20.0275,20.0000,...,18.9175,20.8000,18.0400,17.9950,17.8150,18.4375,18.3725,22.1025,22.2550,21.3275
362,21.6500,22.8825,22.7050,25.3925,20.8925,17.1225,14.2850,13.2600,19.8850,22.9500,...,23.0050,26.7125,23.3825,21.3975,21.7000,21.7250,19.7925,17.5950,13.9525,11.6300
363,10.8800,9.9400,12.9375,17.1025,15.6875,11.6800,8.9700,6.7100,7.1525,9.8150,...,9.1975,12.3525,13.0025,10.7225,9.6675,8.4150,7.7475,7.7475,7.8000,7.2550


In [49]:
T_a_hourly.to_csv('datasets/T_ambient.csv')

In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os

In [ ]:
source_path = 'datasets/citylearn_challenge_2021'
save_path  = 'datasets/data'
def build_data_process(source_path, save_path):
    solar_radiation = pd.DataFrame()
    electricity_demand = pd.DataFrame()
    heating_demand = pd.DataFrame()
    cooling_demand = pd.DataFrame()
    T_a = pd.DataFrame()
    for filename in os.listdir(source_path):
        if filename.startswith('Building') and filename.endswith('.csv'):
            filepath = os.path.join(source_path, filename)
            df = pd.read_csv(filepath)
            electricity_demand[filename] = df['Equipment Electric Power [kWh]']
            heating_demand[filename] = df['DHW Heating [kWh]']
            cooling_demand[filename] = df['Cooling Load [kWh]']
            T_a[filename] = df['Indoor Temperature [C]']
        elif filename == 'weather.csv':
            filepath = os.path.join(source_path, filename)
            df = pd.read_csv(filepath)
            solar_radiation = df['Direct Solar Radiation [W/m2]']
    electricity_demand = pd.DataFrame(electricity_demand.sum(axis=1))
    heating_demand = pd.DataFrame(heating_demand.sum(axis=1))
    cooling_demand = pd.DataFrame(cooling_demand.sum(axis=1))
    T_a = pd.DataFrame(T_a.sum(axis=1)) 

    solar_radiation = pd.DataFrame(solar_radiation.to_numpy().reshape(int(len(solar_radiation)/24), 24))

    electricity_demand = pd.DataFrame(electricity_demand.to_numpy().reshape(int(len(electricity_demand)/24), 24))

    heating_demand = pd.DataFrame(heating_demand.to_numpy().reshape(int(len(heating_demand)/24), 24))

    cooling_demand = pd.DataFrame(cooling_demand.to_numpy().reshape(int(len(cooling_demand)/24), 24))

    T_a = pd.DataFrame(T_a.to_numpy().reshape(int(len(T_a)/24), 24))

    solar_radiation.to_csv(save_path+'solar_radiation.csv')
    electricity_demand.to_csv(save_path+'electricity_demand.csv')
    heating_demand.to_csv(save_path+'heating_demand.csv')
    cooling_demand.to_csv(save_path+'cooling_demand.csv')
    T_a.to_csv(save_path+'T_a.csv')





def price_process(source_path, save_path):
    df = pd.read_csv(source_path)
    electricity_price = df['system_energy_price_da']
    electricity_price = pd.DataFrame(electricity_price.to_numpy().reshape(int(len(electricity_price)/24), 24))
    electricity_price.to_csv(save_path+'electricity_price.csv')




def graph_data_contruct(source_path, save_path):
    price       =   pd.read_csv(source_path+'/electricity_price.csv', index_col=0).to_numpy().reshape(1, -1)
    solar_radiation =   pd.read_csv(source_path+'/solar_radiation.csv', index_col=0).to_numpy().reshape(1, -1)
    e_demand    =   pd.read_csv(source_path+'/electricity_demand.csv', index_col=0).to_numpy().reshape(1, -1)
    h_demand    =   pd.read_csv(source_path+'/heating_demand.csv', index_col=0).to_numpy().reshape(1, -1)
    c_demand    =   pd.read_csv(source_path+'/cooling_demand.csv', index_col=0).to_numpy().reshape(1, -1)
    T_a       =   pd.read_csv(source_path+'/T_a.csv', index_col=0).to_numpy().reshape(1, -1)

    dataset     =   np.concatenate((price, solar_radiation, e_demand, h_demand, c_demand, T_a), axis = 0).T

    np.savetxt('dataset/HIES_data.csv', dataset, delimiter=',')
